## Analizando la Fuerza de Ventas Tipo 1 (Grupal)
* Se eligió este cargo y este producto por ser el de mayo cantidad de colaboradores y por tener mayor rotacion historica desde el 2018

In [1]:
# Librerias Necesarias
import pandas as pd
import altair as alt
#import altair.vegalite.v4_8 as alt
import numpy as np

In [6]:
# Extrae la base desde gsheet

def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

In [7]:
cesados = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
cesados = pd.read_excel(
    cesados,
    usecols='E,I,O,P,S,T,W,X',
    dtype = {'CODIGO':str} #Key debe ser en tipo texto
).rename(
    columns = { #Estandarizo los nombres para no tener problemas
        'Total o Temprana':'TIEMPO_CESE',
        'Producto':'PRODUCTO',
        'TIPO':'TIPO_CESE',
        'MOTIVO PRIMARIO':'MOTIVO_CESE'}
).query('CARGO == "VENTA GRUPAL" and CESE >= "2018-01-01"') #filtro solo el cargo que quiero

* Conociendo a los Cesados

In [9]:
tiempo = cesados.groupby(
            by = 'TIEMPO_CESE', as_index = False
        ).agg(ceses = ('CODIGO', 'count'))

alt.Chart(tiempo).mark_arc(innerRadius = 50).encode(
    theta = alt.Theta(field = 'ceses', type = 'quantitative'),
    color = alt.Color(field = 'TIEMPO_CESE', type = 'nominal')
).properties(
    title = 'Ceses por Tiempo',
    width = 300,
    height = 200)

alt.Chart(...)

In [11]:
tipo = cesados.groupby(
            by = 'TIPO_CESE', as_index = False
        ).agg(ceses = ('CODIGO', 'count'))

alt.Chart(tipo).mark_arc(innerRadius = 50).encode(
    theta = alt.Theta(field = 'ceses', type = 'quantitative'),
    color = alt.Color(field = 'TIPO_CESE', type = 'nominal')
).properties(
    title = 'Ceses por Tipo',
    width = 300,
    height = 200)

alt.Chart(...)

In [18]:
motivos = cesados.groupby(
            by = 'MOTIVO_CESE', as_index = False
        ).agg(
            cesados = ('CODIGO', 'count')
        )

alt.Chart(motivos).mark_bar().encode(
    y = alt.Y(
        'MOTIVO_CESE',
        sort = '-x',
        title = 'Motivo de Ceses',),
    x = alt.X(
        'cesados',
        title = None,
        axis = None),
    color = alt.condition(
        alt.datum.antg_median <= 12, 
        alt.value("#04B2D9"),
        alt.value("#04328C"))
).properties(
    title = 'Motivos de Cese mas frecuentes',
    width = 500,
    height = 390
)

alt.Chart(...)

* Utilizaré las fechas como inicio de mes para estandarizar la fecha de salida e ingreso con otras variables en le futuro
* Hallando la antiguedad de Cese vs Ingreso

In [19]:
cesados['CESE'] = cesados['CESE'].to_numpy().astype('datetime64[M]')
cesados['INGRESO'] = cesados['INGRESO'].to_numpy().astype('datetime64[M]')
cesados['antiguedad_meses'] = (((cesados['CESE'] - cesados['INGRESO']).dt.days)/30).round(0)

In [20]:
cesados.antiguedad_meses.describe()

count    1632.000000
mean       18.468137
std        15.231835
min         0.000000
25%         7.000000
50%        14.000000
75%        25.000000
max        90.000000
Name: antiguedad_meses, dtype: float64

* Analizando la antiguedad de los ceses, si una cese tiene poca antiguedad seria dificil ver

alt.Chart(...)

In [42]:
cesados.query('TIPO_CESE == "INVOLUNTARIA"'
        ).groupby(
            by = 'MOTIVO_CESE', as_index = False
        ).agg(
            cesados = ('CODIGO', 'count')
        ).sort_values(by='cesados', ascending = False)

,MOTIVO_CESE,cesados
7,Fraude,119
11,Termino De Contrato,107
6,Faltas A La Metodología o Política,75
0,Abandono de Trabajo,51
3,Código de Ética,49
9,Programa de Acompañamiento de Mejora,43
2,Bajo desempeño,17
4,Defunción,6
1,Bajo Desempeño,1
5,Falsas Expectativas,1


In [43]:
ceses_antg = cesados.query('TIPO_CESE == "VOLUNTARIA"'
    ).groupby(
    by=['antiguedad_meses'],
    as_index=False
).agg(
    ceses = ('CODIGO', 'count'))

# GRAFICANDO
alt.Chart(ceses_antg).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'ceses',
        title = 'Total Ceses'),
    color = alt.condition(
        alt.datum.antiguedad_meses < 6.0, 
        alt.value('#04B2D9'),
        alt.value('#04328C'))
).properties(
    title = 'Total de Ceses Segun Antiguedad por Producto',
    width = 500,
    height = 250
).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
).resolve_scale(
    y='independent'
)

alt.Chart(...)

In [52]:
cesados.query('TIPO_CESE == "VOLUNTARIA" and antiguedad_meses >= 6'
    ).groupby(
        by = ['MOTIVO_CESE'],
        as_index = False
    ).agg(
        ceses = ('CODIGO', 'count')
    ).sort_values(by='ceses', ascending = False)

,MOTIVO_CESE,ceses
7,Mejor Oferta De Trabajo,226
12,Salud,179
8,Negocio Propio,82
1,Cuidado y atencion de hijos,81
3,Estudios,76
9,No le Gusto el Trabajo,59
0,Cambio De Residencia,52
5,Liderazgo,50
13,Situaciones Personales y Familiares,45
4,Falsas Expectativas,40


In [64]:
#motivo_cese = cesados.query('TIPO_CESE == "VOLUNTARIA" and antiguedad_meses >= 6'
motivo_cese = cesados.query('TIPO_CESE == "VOLUNTARIA"'
    ).groupby(
        by = ['MOTIVO_CESE'],
        as_index = False
    ).agg(
        antg_median = ('antiguedad_meses', 'median')
    )

barra = alt.Chart(motivo_cese).mark_bar().encode(
    y = alt.Y(
        'MOTIVO_CESE',
        sort='x',
        title = 'Motivo de Ceses Voluntarios',),
    x = alt.X(
        'antg_median',
        title = None,
        axis = None),
    color = alt.condition(
        alt.datum.antg_median <= 12, 
        alt.value("#04B2D9"),
        alt.value("#04328C"))
).properties(
    title = 'Mediana Antiguedad por Motivo de Cese - Individual',
    width = 500,
    height = 390
)

texto = barra.mark_text(
    align = 'center',
    baseline ='bottom',
    dy = 8,
    dx = 12,
    color = '#F2F2F2',
    fontSize = 12,
    fontWeight = 'bold'
).encode(
    text = alt.Text('antg_median',format =',.2r')
)

motivos = (barra+texto).properties(
    title = 'Mediana de Antiguedad de Ceses por Motivos Voluntarios',
    width = 400,
    height = 300
)

(barra+texto).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 13,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    font = 'courier',
    color = '#033E8C',
    anchor = 'start'
)

alt.LayerChart(...)

In [56]:
cesados.query('TIPO_CESE == "VOLUNTARIA" and antiguedad_meses >= 6 and MOTIVO_CESE=="Personal Eventual"'
    ).describe()

,antiguedad_meses
count,4.000000
mean,7.000000
std,1.414214
min,6.000000
25%,6.000000
50%,6.500000
75%,7.500000
max,9.000000


In [57]:
cesados.query('TIPO_CESE == "VOLUNTARIA" and antiguedad_meses >= 6 and MOTIVO_CESE=="No le Gusto el Trabajo"'
    ).describe()
    

,antiguedad_meses
count,59.000000
mean,15.745763
std,12.578845
min,6.000000
25%,7.000000
50%,11.000000
75%,17.500000
max,65.000000


In [61]:
no_le_gusto = cesados.query('TIPO_CESE == "VOLUNTARIA" and antiguedad_meses >= 6 and MOTIVO_CESE!="Personal Eventual"'
    ).groupby(
        by = ['antiguedad_meses'],
        as_index = False
    ).agg(
        ceses = ('CODIGO', 'count')
    )
# GRAFICANDO
alt.Chart(no_le_gusto).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=6),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'ceses',
        title = 'Total Ceses'),
    color = alt.condition(
        alt.datum.antiguedad_meses < 6.0, 
        alt.value('#04B2D9'),
        alt.value('#04328C'))
).properties(
    title = 'Total de Ceses Segun Antiguedad por Producto',
    width = 500,
    height = 250
).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
).resolve_scale(
    y='independent'
)

alt.Chart(...)

In [63]:
ceses_antg = cesados.query('TIPO_CESE == "VOLUNTARIA" and antiguedad_meses >= 6 and MOTIVO_CESE!="Personal Eventual"'
    ).groupby(
    by=['antiguedad_meses'],
    as_index=False
).agg(
    ceses = ('CODIGO', 'count'))

# GRAFICANDO
alt.Chart(ceses_antg).mark_bar(
    color = '#04328C'
).encode(
    x = alt.X(
        'antiguedad_meses',
        bin = alt.BinParams(step=3),
        title = 'Antiguedad (meses)'),
    y = alt.Y(
        'ceses',
        title = 'Total Ceses'),
    color = alt.condition(
        alt.datum.antiguedad_meses < 6.0, 
        alt.value('#04B2D9'),
        alt.value('#04328C'))
).properties(
    title = 'Total de Ceses Segun Antiguedad por Producto',
    width = 500,
    height = 250
).configure_view(
    strokeWidth = 0
).configure_axis(
    labelFontSize = 15,
    labelAngle = 0
).configure_title(
    fontSize = 18,
    color = '#033E8C'
).resolve_scale(
    y='independent'
)

alt.Chart(...)